In [1]:
from sqlalchemy import create_engine, text
import pymysql
import pandas as pd

In [35]:
from sklearn import datasets

In [37]:
dir(datasets)

['__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_base',
 '_california_housing',
 '_covtype',
 '_kddcup99',
 '_lfw',
 '_olivetti_faces',
 '_openml',
 '_rcv1',
 '_samples_generator',
 '_species_distributions',
 '_svmlight_format_fast',
 '_svmlight_format_io',
 '_twenty_newsgroups',
 'clear_data_home',
 'dump_svmlight_file',
 'fetch_20newsgroups',
 'fetch_20newsgroups_vectorized',
 'fetch_california_housing',
 'fetch_covtype',
 'fetch_kddcup99',
 'fetch_lfw_pairs',
 'fetch_lfw_people',
 'fetch_olivetti_faces',
 'fetch_openml',
 'fetch_rcv1',
 'fetch_species_distributions',
 'get_data_home',
 'load_boston',
 'load_breast_cancer',
 'load_diabetes',
 'load_digits',
 'load_files',
 'load_iris',
 'load_linnerud',
 'load_sample_image',
 'load_sample_images',
 'load_svmlight_file',
 'load_svmlight_files',
 'load_wine',
 'make_biclusters',
 'make_blobs',
 'make_checkerboard',
 'make_circles',
 'make_class

In [38]:
newsgroup = datasets.fetch_20newsgroups()

In [2]:
driver = 'mysql+pymysql'
user = 'bigdata'
passwd ='bigdata1234'
host = 'localhost'
port = '3306'
db = 'bigdata'
conn = create_engine(f'{driver}://{user}:{passwd}@{host}:{port}/{db}')

In [3]:
qry_stmt = '''
SELECT sepal_length, sepal_width, TARGET
FROM tb_iris
WHERE sepal_length > 5;
'''
pd.read_sql(qry_stmt, conn)

,sepal_length,sepal_width,TARGET
0,5.1,3.5,setosa
1,5.4,3.9,setosa
2,5.4,3.7,setosa
3,5.8,4.0,setosa
4,5.7,4.4,setosa
...,...,...,...
113,6.7,3.0,virginica
114,6.3,2.5,virginica
115,6.5,3.0,virginica
116,6.2,3.4,virginica


In [21]:
iris.loc[iris['sepal_length'] > 5 , ['sepal_length','sepal_width', 'target']]

,sepal_length,sepal_width,target
0,5.1,3.5,setosa
5,5.4,3.9,setosa
10,5.4,3.7,setosa
14,5.8,4.0,setosa
15,5.7,4.4,setosa
...,...,...,...
145,6.7,3.0,virginica
146,6.3,2.5,virginica
147,6.5,3.0,virginica
148,6.2,3.4,virginica


In [22]:
iris.loc[iris.sepal_length > iris.sepal_length.mean(), ['sepal_length','sepal_width', 'target']]

,sepal_length,sepal_width,target
50,7.0,3.2,versicolor
51,6.4,3.2,versicolor
52,6.9,3.1,versicolor
54,6.5,2.8,versicolor
56,6.3,3.3,versicolor
...,...,...,...
145,6.7,3.0,virginica
146,6.3,2.5,virginica
147,6.5,3.0,virginica
148,6.2,3.4,virginica


In [26]:
qry_stmt = '''
SELECT AVG(SEPAL_LENGTH)
FROM tb_iris
'''
pd.read_sql(qry_stmt, conn)

,AVG(SEPAL_LENGTH)
0,5.843333


In [28]:
qry_stmt = '''
SELECT sepal_length, sepal_width, TARGET
FROM tb_iris
WHERE sepal_length > (SELECT avg(sepal_length)
                        FROM tb_iris)
'''
pd.read_sql(qry_stmt, conn)

,sepal_length,sepal_width,TARGET
0,7.0,3.2,versicolor
1,6.4,3.2,versicolor
2,6.9,3.1,versicolor
3,6.5,2.8,versicolor
4,6.3,3.3,versicolor
...,...,...,...
65,6.7,3.0,virginica
66,6.3,2.5,virginica
67,6.5,3.0,virginica
68,6.2,3.4,virginica


In [31]:
qry_stmt = '''
SELECT count(*)
  FROM (SELECT sepal_length, sepal_width, TARGET
            FROM tb_iris
                WHERE sepal_length > (SELECT avg(sepal_length)
                 FROM tb_iris)) A
'''
pd.read_sql(qry_stmt, conn)

,count(*)
0,70


In [33]:
qry_stmt = '''
SELECT avg(sepal_length)
  FROM (SELECT sepal_length, sepal_width, TARGET
            FROM tb_iris
                WHERE sepal_length > (SELECT avg(sepal_length)
                                         FROM tb_iris)) A
'''
pd.read_sql(qry_stmt, conn)

,avg(sepal_length)
0,6.58


In [39]:
qry_stmt = '''
SELECT count(A.TARGET)
  FROM (SELECT TARGET
            FROM tb_iris
                WHERE sepal_length > (SELECT avg(sepal_length)
                                         FROM tb_iris)) A
WHERE A.TARGET = 'VIRGINICA'
'''
pd.read_sql(qry_stmt, conn)

,count(A.TARGET)
0,44


In [41]:
qry_stmt = '''
SELECT A.TARGET, count(A.TARGET)
  FROM (SELECT TARGET
            FROM tb_iris
                WHERE sepal_length > (SELECT avg(sepal_length)
                                         FROM tb_iris)) A
GROUP BY A.TARGET
'''
pd.read_sql(qry_stmt, conn)

,TARGET,count(A.TARGET)
0,versicolor,26
1,virginica,44


In [63]:
qry_stmt = '''
SELECT A.TARGET, A.PETAL_WIDTH + A.SEPAL_WIDTH AS SP_WIDTH, '====='
  FROM (SELECT TARGET, PETAL_WIDTH, SEPAL_WIDTH
            FROM tb_iris
                WHERE sepal_length > (SELECT avg(sepal_length)
                                         FROM tb_iris)) A
WHERE A.TARGET = 'VIRGINICA'
'''
pd.read_sql(qry_stmt, conn)

,TARGET,SP_WIDTH,=====
0,virginica,5.8,=====
1,virginica,5.1,=====
2,virginica,4.7,=====
3,virginica,5.2,=====
4,virginica,5.1,=====
5,virginica,4.7,=====
6,virginica,4.3,=====
7,virginica,6.1,=====
8,virginica,5.2,=====
9,virginica,4.6,=====


In [77]:
# IRIS 테이블에서 TARGET이 SETOSA인 SEPPAL_LENGTH의 평균과
# VIRSISOLOR인 PETAL_WIDTH의 가장 큰 값보다 작은 PETAL_WIDTH를 가진 것 중

# SETOSA의 PETAL_WIDTH, SEPAL_WIDTH의 두 칼럼을 합한 값을 구하고, 그 값의 평균을 구하라


qry_stmt = '''
SELECT AVG(A.PETAL_WIDTH + A.SEPAL_WIDTH)
  FROM (SELECT *
            FROM tb_iris
            WHERE SEPAL_LENGTH > (SELECT AVG(SEPAL_LENGTH)
                                     FROM tb_iris
                                     WHERE TARGET = 'SETOSA')
              AND PETAL_WIDTH < (SELECT MAX(PETAL_WIDTH)
                                      FROM tb_iris
                                      WHERE TARGET = 'VERSICOLOR')) A
'''
pd.read_sql(qry_stmt, conn)

,AVG(A.PETAL_WIDTH + A.SEPAL_WIDTH)
0,4.090278


In [80]:
qry_stmt = '''
SELECT COUNT(DISTINCT SEPAL_LENGTH)
    FROM tb_iris
    GROUP BY TARGET



'''
pd.read_sql(qry_stmt, conn)

,COUNT(DISTINCT SEPAL_LENGTH)
0,15
1,21
2,21


In [81]:
qry_stmt = '''
SELECT SEPAL_LENGTH, COUNT(*)
    FROM tb_iris
    GROUP BY SEPAL_LENGTH



'''
pd.read_sql(qry_stmt, conn)

,SEPAL_LENGTH,COUNT(*)
0,5.1,9
1,4.9,6
2,4.7,2
3,4.6,4
4,5.0,10
5,5.4,6
6,4.4,3
7,4.8,5
8,4.3,1
9,5.8,7


In [90]:
# SEPAL_LENGTH의 UNIQUE한 것의 개수(DISTINCT 사용 X)
# 제일 많이 중복된 SEPAL_LENGTH의 길이

qry_stmt = '''
SELECT COUNT(*)
    FROM (SELECT SEPAL_LENGTH, COUNT(*)
        FROM tb_iris
        GROUP BY SEPAL_LENGTH
        HAVING COUNT(*) = 1) A



'''
pd.read_sql(qry_stmt, conn)

,COUNT(*)
0,9


In [114]:
qry_stmt = '''
SELECT COUNT(*)
FROM(SELECT A.SEPAL_LENGTH, A.CNT
    FROM (SELECT SEPAL_LENGTH, COUNT(*) AS CNT
        FROM tb_iris
        GROUP BY SEPAL_LENGTH) A
WHERE A.CNT = 1) B

'''
pd.read_sql(qry_stmt, conn)

,COUNT(*)
0,9


In [126]:
# 제일 많이 중복된 SEPAL_LENGTH의 길이

qry_stmt = '''
SELECT A.SEPAL_LENGTH, A.CNT
    FROM (SELECT SEPAL_LENGTH, COUNT(*) AS CNT
        FROM tb_iris
        GROUP BY SEPAL_LENGTH) A
WHERE A.CNT  = (SELECT MAX(B.CNT) AS MAX_CNT
                    FROM(SELECT SEPAL_LENGTH,
                            COUNT(*) AS CNT
                            FROM tb_iris
                            GROUP BY SEPAL_LENGTH) B)

'''
pd.read_sql(qry_stmt, conn)

,SEPAL_LENGTH,CNT
0,5.0,10


In [127]:
qry_stmt = '''

SELECT SEPAL_LENGTH, COUNT(*) AS CNT
    FROM tb_iris
    GROUP BY SEPAL_LENGTH
    ORDER BY CNT DESC
LIMIT 1

'''
pd.read_sql(qry_stmt, conn)

,SEPAL_LENGTH,CNT
0,5.0,10


In [25]:
# tb_iris에서 중복이 되는 PETAL_WIDTH값 중 가장 적게 중복되는 것의 개수는?

qry_stmt = '''

SELECT  A.CNT, A.PETAL_WIDTH
    FROM (SELECT COUNT(*) AS CNT, PETAL_WIDTH
            FROM tb_iris
            GROUP BY PETAL_WIDTH) A
    WHERE A.CNT = (SELECT MIN(B.CNT)
                    FROM (SELECT PETAL_WIDTH, COUNT(*) AS CNT
                            FROM tb_iris
                            GROUP BY PETAL_WIDTH
                            HAVING CNT != 1) B)
'''
pd.read_sql(qry_stmt, conn)

,CNT,PETAL_WIDTH
0,2,1.7


In [34]:
qry_stmt = '''

SELECT TITLE, TARGET
    FROM tb_newsgroup
    WHERE title LIKE '%netcom.com%'
'''
pd.read_sql(text(qry_stmt), conn)

,TITLE,TARGET
0,From: nagle@netcom.com (John Nagle)\nSubject: ...,sci.electronics
1,From: noring@netcom.com (Jon Noring)\nSubject:...,sci.med
2,From: bontchev@fbihh.informatik.uni-hamburg.de...,sci.crypt
3,From: cabanrf@wkuvx1.bitnet\nSubject: Re: My B...,rec.sport.baseball
4,From: dashley@wyvern.wyvern.com (Doug Ashley)\...,comp.sys.mac.hardware
...,...,...
441,From: vlad@netcom.com (Vladimir Kuznetsov)\nSu...,talk.politics.misc
442,From: ewm@cbnews.cb.att.com (edward.w.mcfarlan...,rec.motorcycles
443,From: shaig@Think.COM (Shai Guday)\nSubject: R...,talk.politics.mideast
444,From: mathew <mathew@mantis.co.uk>\nSubject: A...,alt.atheism


In [48]:
newsgroup.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [50]:
df_newscategory = newsgroup.target_names
# 'tb_newscategory'
# 'target', 'target_name'

create_stmt = '''
create table tb_newscategory(
target float(10),
target_name varchar(50)
)
'''
conn.execute(text(create_stmt))

In [56]:
insert_stmt = """
insert into tb_newscategory(
target,
target_name
)values
(
:target,
:target_name
)
"""

for k,v in enumerate(df_newscategory):
    dt = {"target":k,"target_name":v}
    conn.execute(text(insert_stmt),**dt)


In [59]:
qry_stmt = '''

SELECT *
    FROM tb_newscategory
'''
pd.read_sql(text(qry_stmt), conn)

,target,target_name
0,0.0,alt.atheism
1,1.0,comp.graphics
2,2.0,comp.os.ms-windows.misc
3,3.0,comp.sys.ibm.pc.hardware
4,4.0,comp.sys.mac.hardware
5,5.0,comp.windows.x
6,6.0,misc.forsale
7,7.0,rec.autos
8,8.0,rec.motorcycles
9,9.0,rec.sport.baseball


In [71]:
qry_stmt = '''

SELECT A.CONTENTS, B.TARGET_NAME
FROM tb_newsgroup A, tb_newscategory B
WHERE A.TARGET = B.TARGET
'''
pd.read_sql(text(qry_stmt), conn)

,CONTENTS,TARGET_NAME
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,sci.space
...,...,...
11309,From: jim.zisfein@factory.com (Jim Zisfein) \n...,sci.med
11310,From: ebodin@pearl.tufts.edu\nSubject: Screen ...,comp.sys.mac.hardware
11311,From: westes@netcom.com (Will Estes)\nSubject:...,comp.sys.ibm.pc.hardware
11312,From: steve@hcrlgw (Steven Collins)\nSubject: ...,comp.graphics


In [100]:
qry_stmt = '''

SELECT COUNT(A.CATEGORY), A.CATEGORY
    FROM (SELECT TARGET,
                LEFT(TARGET_NAME, INSTR(TARGET_NAME, '.')) AS CATEGORY
            FROM tb_newscategory) A
GROUP BY CATEGORY

'''
pd.read_sql(text(qry_stmt), conn)

,COUNT(A.CATEGORY),CATEGORY
0,1,alt.
1,5,comp.
2,1,misc.
3,3,rec.
4,4,sci.
5,1,soc.
6,4,talk.


In [101]:
qry_stmt = '''
SELECT TARGET, LEFT(TARGET_NAME, INSTR(TARGET_NAME, '.')) AS CATEGORY
        FROM tb_newscategory
'''
pd.read_sql(text(qry_stmt), conn)

,TARGET,CATEGORY
0,0.0,alt.
1,1.0,comp.
2,2.0,comp.
3,3.0,comp.
4,4.0,comp.
5,5.0,comp.
6,6.0,misc.
7,7.0,rec.
8,8.0,rec.
9,9.0,rec.


In [99]:
conn.execute(text('DELETE FROM tb_newscategory WHERE TARGET = 10'))

In [104]:
qry = '''

SELECT COUNT(*)
    FROM(SELECT A.CONTENTS, B.TARGET_NAME
            FROM tb_newsgroup A, tb_newscategory B
            WHERE A.TARGET = B.TARGET) A
'''


pd.read_sql(text(qry), conn)

,COUNT(*)
0,10714


In [120]:
qry = '''

SELECT A.TARGET, B.TARGET, IFNULL(B.TARGET_NAME, '없음')
    FROM tb_newsgroup A LEFT OUTER JOIN tb_newscategory B ON (A.TARGET = B.TARGET)
   WHERE A.TARGET = 10 OR A.TARGET = 20
'''


pd.read_sql(text(qry), conn)

,TARGET,TARGET,"IFNULL(B.TARGET_NAME, '없음')"
0,10.0,None,없음
1,10.0,None,없음
2,10.0,None,없음
3,10.0,None,없음
4,10.0,None,없음
...,...,...,...
595,10.0,None,없음
596,10.0,None,없음
597,10.0,None,없음
598,10.0,None,없음


In [153]:
## tb_newsgroup에서 target - code이고, code는 소분류임
## 대분류는 tb_newscategory에 있음 xxx.
## tb_newsgroup와 tb_newscategory를 join해서 대분류별로 count하고
## 대분류의 카운트된 숫자대로 큰 순서로 정렬하자
## 만약, tb_newsgroup의 target이 tb_newscategory에 없으면 unknown으로 표시하여
## 대분류 카테고리에 적용 후
## 카운트에 반영하세요

qry = '''

SELECT COUNT(*), 대분류
FROM (SELECT IFNULL(LEFT(TARGET_NAME, INSTR(B.TARGET_NAME, '.')-1), 'UNKNOWN') AS 대분류
        FROM tb_newsgroup A LEFT JOIN tb_newscategory B ON (A.TARGET = B.TARGET)) C
GROUP BY C.대분류
ORDER BY COUNT(*) DESC
'''


pd.read_sql(text(qry), conn)

,COUNT(*),대분류
0,2936,comp
1,2373,sci
2,1952,talk
3,1789,rec
4,600,UNKNOWN
5,599,soc
6,585,misc
7,480,alt


In [159]:
## 소분류 카테고리(0~20)가 7번보다 큰 대분류 중 CONTENTS 내용에 EDU가 들어간 CONTENTS의
## 대분류별 개수


qry = '''
SELECT 대분류, COUNT(*) AS 개수
FROM (SELECT IFNULL(LEFT(TARGET_NAME, INSTR(B.TARGET_NAME, '.')-1), 'UNKNOWN') AS 대분류,
             A.TARGET, A.CONTENTS
        FROM tb_newsgroup A LEFT JOIN tb_newscategory B ON (A.TARGET = B.TARGET)
       WHERE A.TARGET > 7
         AND A.CONTENTS LIKE '%edu%') C
GROUP BY C.대분류
ORDER BY COUNT(*) DESC;
'''


pd.read_sql(text(qry), conn)

,대분류,개수
0,sci,1547
1,talk,1469
2,rec,851
3,soc,473
4,UNKNOWN,368


In [167]:
# tb_wine 데이터를 사용하여 color_intensity 컬럼 옆에다 평균값을 붙여보자
# 스칼라 서브쿼리(SELECT절의 서브쿼리)

qry = '''
SELECT color_intensity, (SELECT avg(color_intensity) FROM tb_wine)
    FROM tb_wine
    '''


pd.read_sql(text(qry), conn)

,color_intensity,(SELECT avg(color_intensity) FROM tb_wine)
0,5.64,5.05809
1,4.38,5.05809
2,5.68,5.05809
3,7.80,5.05809
4,4.32,5.05809
...,...,...
173,7.70,5.05809
174,7.30,5.05809
175,10.20,5.05809
176,9.30,5.05809


In [171]:
# color_intensity 평균 표준편차


qry = '''
SELECT color_intensity, 
        (SELECT AVG(color_intensity) FROM tb_wine) AS MEAN,
        (SELECT STD(color_intensity) FROM tb_wine) AS STD,
        (SELECT SUM(color_intensity) FROM tb_wine) AS SUM
FROM tb_wine;
'''
pd.read_sql(text(qry), conn)

,color_intensity,MEAN,STD,SUM
0,5.64,5.05809,2.311765,900.339998
1,4.38,5.05809,2.311765,900.339998
2,5.68,5.05809,2.311765,900.339998
3,7.80,5.05809,2.311765,900.339998
4,4.32,5.05809,2.311765,900.339998
...,...,...,...,...
173,7.70,5.05809,2.311765,900.339998
174,7.30,5.05809,2.311765,900.339998
175,10.20,5.05809,2.311765,900.339998
176,9.30,5.05809,2.311765,900.339998


In [179]:
qry = '''
SELECT CONCAT(COL_1,',', COL_2)
    FROM tb_digit;
'''
pd.read_sql(text(qry), conn)

,"CONCAT(COL_1,',', COL_2)"
0,"0,5"
1,"0,0"
2,"0,0"
3,"0,7"
4,"0,0"
...,...
1792,"0,4"
1793,"0,6"
1794,"0,1"
1795,"0,2"
